In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
import matplotlib.pyplot as plt
import math
import tensorflow.keras.backend as K
from functools import partial
import tensorflow_addons as tfa

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')
os.system('pip install ../input/tf2cvmodels/TensorFlow-ResNets-master/ -q --no-deps')

In [ ]:
from tf2_resnets import models

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [512, 512]

In [ ]:
# https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
# https://towardsai.net/p/machine-learning/building-complex-image-augmentation-pipelines-with-tensorflow-bed1914278d2

def data_augment(image):
#     image = tf.image.random_flip_left_right(image)
    
#     p_rotation = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
#     p_shear = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    # Shear
#     if p_shear > .2:
#         if p_shear > .6:
#             image = transform_shear(image, HEIGHT, shear=20.)
#         else:
#             image = transform_shear(image, HEIGHT, shear=-20.)
            
    # Rotation
#     if p_rotation > .2:
#         if p_rotation > .6:
#             image = transform_rotation(image, HEIGHT, rotation=45.)
#         else:
#             image = transform_rotation(image, HEIGHT, rotation=-45.)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.6)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.7)
        else:
            image = tf.image.central_crop(image, central_fraction=.8)
    elif p_crop > .4:
        crop_size = tf.random.uniform([], int(512*.6), 512, dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, 3])
            
    image = tf.image.resize(image, size=[512, 512])
    return image

In [ ]:
from albumentations import (
    Compose, RandomBrightness, HorizontalFlip, VerticalFlip, Flip, CenterCrop,
    Rotate, ShiftScaleRotate, RandomResizedCrop)

TTA1 = Compose([
    HorizontalFlip(p=1)
])

TTA2 = Compose([
    VerticalFlip(p=1)
])

TTA3 = Compose([
    Flip(p=1),
#     Rotate(limit=90, p=1)
])

TTA4 = Compose([
#     Flip(p=1),
#     HorizontalFlip(p=0.5),
    CenterCrop(256, 256, p=1.0),
#     ShiftScaleRotate(p=1),
#     Rotate(limit=90, p=1)
])

TTA5 = Compose([
#     Flip(p=1),
#     HorizontalFlip(p=0.5),
    RandomResizedCrop(512, 512, p=1),
#     ShiftScaleRotate(p=1),
#     Rotate(limit=90, p=1)
])

TTA6 = Compose([
#     Flip(p=1),
    HorizontalFlip(p=1),
    CenterCrop(256, 256, p=1.0),
    ShiftScaleRotate(p=1),
#     Rotate(limit=90, p=1)
])

TTA7 = Compose([
#     Flip(p=1),
    HorizontalFlip(p=1),
    RandomResizedCrop(512, 512, p=1),
    ShiftScaleRotate(p=1),
#     Rotate(limit=90, p=1)
])

TTA8 = Compose([
    Rotate(limit=90, p=1)
])

TTA9 = Compose([
    ShiftScaleRotate(p=1),
])

# TTA10 = Compose([
# #     Flip(p=1),
#     HorizontalFlip(p=0.5),
#     RandomResizedCrop(512, 512, p=1),
#     ShiftScaleRotate(p=1),
# #     Rotate(limit=90, p=1)
# ])

# tta = [TTA1, TTA2, TTA3, TTA4, TTA5, TTA6, TTA7, TTA8, TTA9]
tta = [TTA1, TTA2, TTA4, TTA5]
# tta = [TTA1, TTA2, TTA3]

In [ ]:
def decode_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, (512, 512))
    img = tf.reshape(img, [512, 512, 3])
    return img

def aug_fn(image, img_size, tta_num):
    data = {"image":image}
    aug_data = tta[tta_num](**data)
    aug_img = aug_data["image"]
#     aug_img = tf.cast(aug_img/255.0, tf.float32)
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_data(image, img_size, tta_num):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size, tta_num], Tout=tf.float32)
    return aug_img

def get_test_dataset(dataset, tta_num, ordered=False, do_aug=True):
#     dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    if do_aug:
#         dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)
#         dataset = dataset.map(transform, num_parallel_calls=AUTOTUNE)
        dataset = dataset.map(partial(process_data, img_size=512, tta_num=tta_num), num_parallel_calls=AUTOTUNE)
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
predictions = pd.DataFrame()
predictions['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))

TEST_FILENAMES = tf.io.gfile.glob("/kaggle/input/cassava-leaf-disease-classification/test_images/*.jpg")
images = tf.constant(TEST_FILENAMES)
dataset = tf.data.Dataset.from_tensor_slices(images)
dataset = dataset.map(decode_image)

In [ ]:
class CategoricalFocalLossLabelSmoothing(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, ls=0.1, classes=5.0):
        super(CategoricalFocalLossLabelSmoothing, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.ls = ls
        self.classes = classes
        
    def focal_loss(self, y_true, y_pred, gamma, alpha, ls, classes):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        #label smoothing
        y_pred_ls = (1 - ls) * y_pred + ls / classes
        # Clip the prediction value
        y_pred_ls = K.clip(y_pred_ls, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -y_true*K.log(y_pred_ls)
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = alpha * y_true * K.pow((1-y_pred_ls), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
        
    def call(self, y_true, y_pred):
        return self.focal_loss(y_true, y_pred, gamma=self.gamma, alpha=self.alpha, ls=self.ls, classes=self.classes)

In [ ]:
from keras import applications
import efficientnet.keras as efn
from efficientnet.keras import EfficientNetB3

In [ ]:
preds = []
models1 = tf.io.gfile.glob("../input/models/model*.h5")
models1 = [keras.models.load_model(model, custom_objects={'CategoricalFocalLossLabelSmoothing':CategoricalFocalLossLabelSmoothing}, compile=False) for model in models1]
for model in models1:
    model.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-3), loss = CategoricalFocalLossLabelSmoothing(gamma=-0.05, alpha=0.1, ls=0.3, classes=5.0), metrics=['categorical_accuracy'])

models2 = tf.io.gfile.glob("../input/models/resnetmodel*.h5")
models2 = [keras.models.load_model(model) for model in models2]

models1.extend(models2)
    
    
for model in models1:
    test_ds = get_test_dataset(dataset, tta_num=None, do_aug=False)
    pred = model.predict(test_ds)
    preds.append(pred)
    
tta_steps = 5
for tta_num in range(len(tta)):
    test_ds = get_test_dataset(dataset, tta_num=tta_num, do_aug=True)
    for model in models1:
        pred = model.predict(test_ds)
        preds.append(pred)
    
        

y_pred = np.mean(preds, axis=0)

In [ ]:
predictions['label'] = np.argmax(y_pred, axis=1)

In [ ]:
predictions.to_csv('submission.csv', index=False)